# Generic Rule Induction Notebook

Continuoulsy refined.

### README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment

In [18]:
import pandas as pd
import numpy as np
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
# from ripper.Ripper import Ripper
import time
import warnings

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
from aix360.algorithms.rbm import FeatureBinarizer, FeatureBinarizerFromTrees

# from explainer import Explainer

# TODO make imports dependent on CONFIG
# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Config

In [19]:

CONFIG = { 
    'NAME' : 'bike-demand', 
    'DATA_SET': 'data/SeoulBikeData.csv',
    'DATA_TYPES': {'Rented Bike Count': float, 'Hour': float, 'Humidity': float, 'Visibility (10m)': float, 'RatePlan': str},
    'DROP': ['Date'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3,
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'Rented Bike Count',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Dew point temperature(C)',
    'POS_CLASS': None
     }
CONFIG = { 
    'NAME' : 'continuous-churn', 
    'DATA_SET': 'data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'CHURN', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'GLRM',
    'TARGET_LABEL': 'pChurn',
    'TYPE' : 'CONTINUOUS',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': None
     }
CONFIG = { 
    'NAME' : 'binary-churn-quantile-brcg', 
    'DATA_SET': '../data/churn_prob_out_35.csv',
    'DATA_TYPES': {'Children': float, 'RatePlan': str},
    'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'],
    'MODE': 'PREDICTIVE',
    'TRAIN_TEST_SPLIT': 0.3, # 'FIXED' for using 'is_test_set'
    'BINARIZER': 'QUANTILE',
    'ALGO': 'BRCG',
    'TARGET_LABEL': 'CHURN',
    'TYPE' : 'BINARY',
    'EXAMPLE_FEATURE' : 'Est Income',
    'POS_CLASS': 'T'
     }


print('Proceed with configuration:', CONFIG['NAME'])

Proceed with configuration: binary-churn-quantile-brcg


### Data

In [20]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG':
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert) 
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CHURN       1799 non-null   int64  
 1   Gender      1799 non-null   object 
 2   Status      1799 non-null   object 
 3   Children    1799 non-null   float64
 4   Est Income  1799 non-null   float64
 5   Car Owner   1799 non-null   object 
 6   Age         1799 non-null   float64
 7   Paymethod   1799 non-null   object 
 8   Usage       1799 non-null   float64
 9   RatePlan    1799 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 140.7+ KB


,CHURN,Gender,Status,Children,Est Income,Car Owner,Age,Paymethod,Usage,RatePlan
0,1,F,S,1.0,38000.00,N,24.393333,CC,229.64,3
1,0,M,M,2.0,29616.00,N,49.426667,CH,75.29,2
2,0,M,M,0.0,19732.80,N,50.673333,CC,47.25,3
3,0,M,S,2.0,96.33,N,56.473333,CC,59.01,1
4,0,F,M,2.0,52004.80,N,25.140000,CH,28.14,1


In [21]:
df[CONFIG['TARGET_LABEL']].describe()

count    1799.000000
mean        0.401890
std         0.490416
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: CHURN, dtype: float64

### Train, Test Split

In [22]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (1259, 9) (1259,)
Test: (540, 9) (540,)


### Reference Performance

In [23]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [24]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=25,negations=True) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info(verbose=True)
x_train_bin.head()
x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1259 entries, 513 to 1126
Data columns (total 178 columns):
 #   Column                                Dtype
---  ------                                -----
 0   (Gender, , )                          int64
 1   (Gender, not, )                       int64
 2   (Status, ==, D)                       int64
 3   (Status, ==, M)                       int64
 4   (Status, ==, S)                       int64
 5   (Status, !=, D)                       int64
 6   (Status, !=, M)                       int64
 7   (Status, !=, S)                       int64
 8   (Children, <=, 0.0)                   int64
 9   (Children, <=, 1.0)                   int64
 10  (Children, >, 0.0)                    int64
 11  (Children, >, 1.0)                    int64
 12  (Est Income, <=, 2067.82)             int64
 13  (Est Income, <=, 5252.29)             int64
 14  (Est Income, <=, 8533.283076923057)   int64
 15  (Est Income, <=, 13101.023076923077)  int64
 16  (Es

operation           <=                                                      \
value     2067.820000  5252.290000  8533.283077  13101.023077 15989.300000   
513                  0            0            0            0            0   
81                   0            0            0            1            1   
596                  0            0            1            1            1   
380                  0            0            0            0            0   
1523                 0            0            1            1            1   
84                   0            0            0            0            0   
1666                 0            0            1            1            1   
1497                 0            0            0            0            0   
1200                 0            0            0            0            0   
1381                 0            0            0            0            0   

operation                                                                   \
value     19749.300000 23321.230769 28787.400000 35000.000000 39461.000000   
513                  0            0            0            0            0   
81                   1            1            1            1            1   
596                  1            1            1            1            1   
380                  0            0            1            1            1   
1523                 1            1            1            1            1   
84                   0            0            0            0            0   
1666                 1            1            1            1            1   
1497                 0            0            0            0            0   
1200                 0            0            1            1            1   
1381                 1            1            1            1            1   

operation  ...            >                                         \
value      ... 66921.400000 69486.800000 73400.000000 76791.200000   
513        ...            1            1            1            1   
81         ...            0            0            0            0   
596        ...            0            0            0            0   
380        ...            0            0            0            0   
1523       ...            0            0            0            0   
84         ...            0            0            0            0   
1666       ...            0            0            0            0   
1497       ...            1            1            1            0   
1200       ...            0            0            0            0   
1381       ...            0            0            0            0   

operation                                                                   \
value     80000.000000 83892.538462 87578.200000 91150.107692 93454.300000   
513                  1            1            1            1            1   
81                   0            0            0            0            0   
596                  0            0            0            0            0   
380                  0            0            0            0            0   
1523                 0            0            0            0            0   
84                   0            0            0            0            0   
1666                 0            0            0            0            0   
1497                 0            0            0            0            0   
1200                 0            0            0            0            0   
1381                 0            0            0            0            0   

operation               
value     96223.400000  
513                  1  
81                   0  
596                  0  
380                  0  
1523                 0  
84                   0  
1666                 0  
1497                 0  
1200                 0  
1381                 0  

[10 rows x 50 columns]

### Rule Induction

In [25]:
start_time = time.time()

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    print('TODO plugin RIPPER here')
    # Ripper(d=2, k=2, pruning_threshold=50, mode='python')
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression(lambda0=0.0005,lambda1=0.0001)
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('training time: ' + str(end_time - start_time))

Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Iteration: 1, Objective: 0.2903
Iteration: 2, Objective: 0.2594
Iteration: 3, Objective: 0.1756
Iteration: 4, Objective: 0.1361
Iteration: 5, Objective: 0.1103
Iteration: 6, Objective: 0.1088
Iteration: 7, Objective: 0.0947
Iteration: 8, Objective: 0.0924
Iteration: 9, Objective: 0.0917
training time: 7.3769919872283936


### Evaluation

In [26]:
if CONFIG['TYPE'] == 'BINARY':
    # compute predictive accuracy
    # print(estimator.dnf_rules.items())
    y_pred = estimator.predict(x_test_bin)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy)
    # print("Accuracy: %.2f%%" % (accuracy * 100.0))
    # print('Accuracy 2:', accuracy_score(y_test, estimator.predict(x_test)))
    # print('Accuracy 3', estimator.score(x_test, y_test))
    # print('Accuracy 4', estimator.score(x_test, y_test))

    # res = estimator.score(x_test, y_test) # this score function is defect
    # print('acc: ' + str(res))
    # print('rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
    # estimator.rule_map
    # print(estimator.rule_list_to_pretty_string())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    explainer.explain()
    # explanation.style

Accuracy: 0.9111111111111111
Number of rules: 7
['Est Income <= 8533.28 AND Age <= 15.38', 'Gender not  AND Status == S AND Est Income <= 5252.29', 'Gender   AND Children <= 1.00 AND Est Income > 58581.65 AND Age > 39.49', 'Est Income <= 49072.00 AND Age <= 22.76 AND Age > 15.38 AND Usage > 74.78', 'Gender not  AND Status != M AND Est Income <= 43440.50 AND Paymethod != CH AND Usage > 12.20', 'Children <= 1.00 AND Est Income <= 49072.00 AND Age <= 25.03 AND Usage <= 215.36 AND Usage > 33.44', 'Est Income > 58581.65 AND Paymethod == CH AND Usage > 33.44 AND RatePlan != 2 AND RatePlan != 3']
